In [40]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.utils import shuffle
from sklearn.svm import SVC
import sklearn
from sklearn.metrics import confusion_matrix,classification_report
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC
from catboost import CatBoostClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.preprocessing import MultiLabelBinarizer
from xgboost import XGBClassifier as xgb
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold
import lightgbm as lgb
import seaborn as sns
from lightgbm import LGBMClassifier
%matplotlib inline
pd.set_option('display.max_columns', None)

df_train=pd.read_csv('train_fNxu4vz.csv')
df_test=pd.read_csv('test_fjtUOL8.csv')
df_submission=pd.read_csv('sample_submission_HSqiq1Q.csv')


In [2]:
for column in ['Months_Since_Deliquency']:
    df_train[column].fillna(0, inplace=True)
    df_test[column].fillna(0, inplace=True)


In [3]:
for column in ['Length_Employed', 'Home_Owner']:
    df_train[column].fillna(df_train[column].mode()[0], inplace=True)
    df_test[column].fillna(df_test[column].mode()[0], inplace=True)

In [4]:
for column in ['Annual_Income']:
    df_train[column].fillna(df_train[column].mean(), inplace=True)
    df_test[column].fillna(df_test[column].mean(), inplace=True)

In [5]:
gender =pd.get_dummies(df_train['Gender'])
gendert = pd.get_dummies(df_test['Gender'])

In [6]:
income =pd.get_dummies(df_train['Income_Verified'])
income1 =pd.get_dummies(df_test['Income_Verified'])


In [7]:
House_Owned =pd.get_dummies(df_train['Home_Owner'])
House_Owned1 =pd.get_dummies(df_test['Home_Owner'])


In [8]:
Loan_Purpose =pd.get_dummies(df_train['Purpose_Of_Loan'])
Loan_Purpose1 =pd.get_dummies(df_test['Purpose_Of_Loan'])


In [9]:
Employed_Duration =pd.get_dummies(df_train['Length_Employed'])
Employed_Duration1 =pd.get_dummies(df_test['Length_Employed'])


In [10]:
df_train=pd.concat([df_train,Loan_Purpose,House_Owned,income,gender,Employed_Duration],axis=1)
df_test=pd.concat([df_test,Loan_Purpose1,House_Owned1,income1,gendert,Employed_Duration1],axis=1)

In [11]:
df_train

,Loan_ID,Loan_Amount_Requested,Length_Employed,Home_Owner,Annual_Income,Income_Verified,Purpose_Of_Loan,Debt_To_Income,Inquiries_Last_6Mo,Months_Since_Deliquency,Number_Open_Accounts,Total_Accounts,Gender,Interest_Rate,car,credit_card,debt_consolidation,educational,home_improvement,house,major_purchase,medical,moving,other,renewable_energy,small_business,vacation,wedding,Mortgage,None,Other,Own,Rent,VERIFIED - income,VERIFIED - income source,not verified,Female,Male,1 year,10+ years,2 years,3 years,4 years,5 years,6 years,7 years,8 years,9 years,< 1 year
0,10000001,7000.0,< 1 year,Rent,68000.000000,not verified,car,18.37,0,0.0,9,14,Female,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,1
1,10000002,30000.0,4 years,Mortgage,73331.159434,VERIFIED - income,debt_consolidation,14.93,0,17.0,12,24,Female,3,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0
2,10000003,24725.0,7 years,Mortgage,75566.400000,VERIFIED - income source,debt_consolidation,15.88,0,0.0,12,16,Male,3,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0
3,10000004,16000.0,< 1 year,Mortgage,56160.000000,VERIFIED - income source,debt_consolidation,14.34,3,0.0,16,22,Male,3,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1
4,10000005,17000.0,8 years,Own,96000.000000,VERIFIED - income source,debt_consolidation,22.17,1,0.0,19,30,Female,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
164304,10164305,20725.0,10+ years,Rent,73331.159434,VERIFIED - income source,credit_card,13.56,0,0.0,13,16,Male,2,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0
164305,10164306,6000.0,8 years,Own,73331.159434,VERIFIED - income source,small_business,9.12,0,36.0,13,30,Male,2,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0
164306,10164307,20250.0,10+ years,Mortgage,73331.159434,VERIFIED - income,credit_card,18.17,1,0.0,17,30,Male,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0
164307,10164308,10000.0,4 years,Rent,73331.159434,not verified,major_purchase,18.75,1,48.0,16,62,Female,3,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0


In [12]:
df_train.drop(['Loan_ID','Purpose_Of_Loan','Gender','Home_Owner','Income_Verified','Length_Employed'],axis=1,inplace=True)
df_test.drop(['Loan_ID','Purpose_Of_Loan','Gender','Home_Owner','Income_Verified','Length_Employed'],axis=1,inplace=True)

In [13]:
df_train

,Loan_Amount_Requested,Annual_Income,Debt_To_Income,Inquiries_Last_6Mo,Months_Since_Deliquency,Number_Open_Accounts,Total_Accounts,Interest_Rate,car,credit_card,debt_consolidation,educational,home_improvement,house,major_purchase,medical,moving,other,renewable_energy,small_business,vacation,wedding,Mortgage,None,Other,Own,Rent,VERIFIED - income,VERIFIED - income source,not verified,Female,Male,1 year,10+ years,2 years,3 years,4 years,5 years,6 years,7 years,8 years,9 years,< 1 year
0,7000.0,68000.000000,18.37,0,0.0,9,14,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,1
1,30000.0,73331.159434,14.93,0,17.0,12,24,3,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0
2,24725.0,75566.400000,15.88,0,0.0,12,16,3,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0
3,16000.0,56160.000000,14.34,3,0.0,16,22,3,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1
4,17000.0,96000.000000,22.17,1,0.0,19,30,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
164304,20725.0,73331.159434,13.56,0,0.0,13,16,2,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0
164305,6000.0,73331.159434,9.12,0,36.0,13,30,2,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0
164306,20250.0,73331.159434,18.17,1,0.0,17,30,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0
164307,10000.0,73331.159434,18.75,1,48.0,16,62,3,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0


In [14]:
target_map = {1:0, 2:1, 3:2}
target_map_inverse = {0:1, 1:2, 2:3}

In [15]:
df_train["Interest_Rate"] = df_train["Interest_Rate"].map(target_map)

In [31]:

features = [col for col in df_train.columns if col not in ["Interest_Rate"]]
target = df_train["Interest_Rate"]
df_train.drop(['Interest_Rate'],axis=1,inplace=True)

In [32]:
df_train

,Loan_Amount_Requested,Annual_Income,Debt_To_Income,Inquiries_Last_6Mo,Months_Since_Deliquency,Number_Open_Accounts,Total_Accounts,car,credit_card,debt_consolidation,educational,home_improvement,house,major_purchase,medical,moving,other,renewable_energy,small_business,vacation,wedding,Mortgage,None,Other,Own,Rent,VERIFIED - income,VERIFIED - income source,not verified,Female,Male,1 year,10+ years,2 years,3 years,4 years,5 years,6 years,7 years,8 years,9 years,< 1 year
0,7000.0,68000.000000,18.37,0,0.0,9,14,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,1
1,30000.0,73331.159434,14.93,0,17.0,12,24,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0
2,24725.0,75566.400000,15.88,0,0.0,12,16,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0
3,16000.0,56160.000000,14.34,3,0.0,16,22,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1
4,17000.0,96000.000000,22.17,1,0.0,19,30,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
164304,20725.0,73331.159434,13.56,0,0.0,13,16,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0
164305,6000.0,73331.159434,9.12,0,36.0,13,30,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0
164306,20250.0,73331.159434,18.17,1,0.0,17,30,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0
164307,10000.0,73331.159434,18.75,1,48.0,16,62,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0


In [33]:
# Scaling the Train and Test feature set 
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(df_train)
X_test_scaled = scaler.transform(df_test)

## Cross Validaton


In [43]:
model = LGBMClassifier(
    objective='binary',
    boosting='gbdt',
    learning_rate = 0.05,
    max_depth = 8,
    num_leaves = 80,
    n_estimators = 400,
    bagging_fraction = 0.8,
    feature_fraction = 0.9)



In [44]:
model.fit(X_train_scaled,target)

LGBMClassifier(bagging_fraction=0.8, boosting='gbdt', boosting_type='gbdt',
               class_weight=None, colsample_bytree=1.0, feature_fraction=0.9,
               importance_type='split', learning_rate=0.05, max_depth=8,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=400, n_jobs=-1, num_leaves=80, objective='binary',
               random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [45]:
predictions_test = model.predict(X_test_scaled)

In [46]:
df_submission["Interest_Rate"] = predictions_test
df_submission["Interest_Rate"] = df_submission["Interest_Rate"].map(target_map_inverse)

In [47]:
df_submission.to_csv('solution.csv',index = False)

NameError: name 'X' is not defined